<center>
    <span style="color:#1E90FF">
    <h1>
        Practical Web Mining for Business Applications
    </h1>
    <h2>
        Document Classification
    </h2>
    </span>
</center>

Definition of path for all the program

In [2]:
path = 'E:\corpus'

<center style="color:#1E90FF">
    <h1>
        1. Text Pre-Processing and Exploration
    </h1>
</center>

First lets load the corpus and split it to train and test

In [45]:
import sklearn.datasets as sk

train_set = sk.load_files(path + "\\training", description=None, categories=None, 
                        load_content=True, shuffle=True, encoding=None, decode_error='strict', random_state=0)
test_set = sk.load_files(path + "\\test", description=None, categories=None, 
                       load_content=True, shuffle=True, encoding=None, decode_error='strict', random_state=0)

# I wish to load the train data again to display differences later on
backup_train = sk.load_files(path + "\\training", description=None, categories=None, 
                        load_content=True, shuffle=True, encoding=None, decode_error='strict', random_state=0)

print("Corupus loaded successfully!")


Corupus loaded successfully!


# Clean and normalize the text

First, lets define some global variables and imports

In [49]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import defaultdict
import string

term_per_category = defaultdict(lambda: defaultdict(int))
stop_words = stopwords.words('english') + list(string.punctuation)


The following loop iterates the files clean it, by using: tokenization, lower case, stop words removal, stemming).

In addition, we will loop the text and count the distrebution of terms in each category into 'term_per_category'

In [50]:
def clean_text(data):
    for i, target in zip(range(len(data.data)), data.target):
        #------------TO LOWER CASE-------------
        file = data.data[i].decode("utf-8").lower()
        #------------TOKENIZE-------------
        word_tokens = word_tokenize(file)
        #------------REMOVE STOP WORDS-------------
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                    filtered_sentence.append(w)
        #------------STEMMING WITH PORTER STEMMER-------------
        ps = PorterStemmer()
        stemmedFile = []
        for word in filtered_sentence:
            for w in word.split(" "):
                stem = ps.stem(w)
                stemmedFile.append(stem)
                #COUNT THE TERMS PER CATEGORY
                term_per_category[train_set.target_names[target]][word] += 1
        #------------PUT FILE BACK-------------
        data.data[i] = ' '.join(stemmedFile)

Now, lets use the methode above to clean the Train and the Test

In [55]:
clean_text(train_set)
clean_text(test_set)

##### Differences between the train data before and after normalizing and cleaning the files

#### Before:

In [56]:
backup_train.data[0]

"Management of Cushing's syndrome secondary to adrenal adenoma during pregnancy.\n This report discusses our experience with two patients who had unilateral adrenalectomy during pregnancy as treatment for Cushing's syndrome secondary to an adrenal adenoma.\n Previously only five patients with this clinical problem who underwent unilateral adrenalectomy during pregnancy had been reported.\n We have reviewed the world literature on Cushing's syndrome in pregnancy secondary to an adrenal adenoma.\n A total of 19 patients who had unilateral adrenalectomy for this problem after the completion of pregnancy were identified.\n The review of world literature and the two patients who are the subject of this report were the basis of our analysis of fetal death, neonatal complications, and maternal complications in seven pregnancies during which unilateral adrenalectomy was performed (group 1) compared to the 19 pregnancies that were associated with unilateral adrenalectomy at the completion of pr

#### After:

In [57]:
train_set.data[0]

u"manag cush 's syndrom secondari adren adenoma pregnanc report discuss experi two patient unilat adrenalectomi pregnanc treatment cush 's syndrom secondari adren adenoma previou five patient clinic problem underw unilat adrenalectomi pregnanc report review world literatur cush 's syndrom pregnanc secondari adren adenoma total 19 patient unilat adrenalectomi problem complet pregnanc identifi review world literatur two patient subject report basi analysi fetal death neonat complic matern complic seven pregnanc unilat adrenalectomi perform group 1 compar 19 pregnanc associ unilat adrenalectomi complet pregnanc group 2 seven pregnanc group 1 one fetal death neonat complic occur fetal death neonat complic occur 12 19 pregnanc group 2. four seven mother group 1 complic 16 19 mother group 2 complic studi suggest adrenalectomi pregnanc consid therapeut option manag cush 's syndrom secondari adren cortic adenoma"

## Explore the Dataset

Display the amount of categories, and their names:

In [58]:
target_names = train_set.target_names
categories = test_set.target_names

print "%d categories" % len(categories)
print
print "Categories names: ",categories

23 categories

Categories names:  ['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23']


### Number of documents per category

Using pandas and default dictionary:

In [59]:
from collections import defaultdict
import pandas as pd

#iterate all the targeted files
files_in_categories = defaultdict(int)
for target in train_set.target:
    files_in_categories[train_set.target_names[target]] += 1

df = pd.DataFrame(files_in_categories.items(), columns=['Categoty id', '# of documents'])

df

,Categoty id,# of documents
0,C19,191
1,C18,388
2,C13,281
3,C12,491
4,C11,162
5,C10,621
6,C17,295
7,C16,200
8,C15,215
9,C14,1249


### Terms distrebution category

After adding all the terms into 'term_per_category', lets sort it and print it's top 10 words

In [60]:
import operator

for dic in term_per_category:
    sorted_x = sorted(term_per_category[dic].items(), key=lambda kv: kv[1] , reverse=True)
    df = pd.DataFrame(sorted_x[:10], columns=['Categoty id', 'top terms']) 
    print '=' * 80
    print 'Category ', dic
    print df 
 

Category  C19
  Categoty id  top terms
0     patient       2279
1      diabet       1133
2     thyroid        814
3         +/-        798
4        less        664
5       level        651
6           p        587
7       studi        572
8      normal        508
9       group        472
Category  C18
  Categoty id  top terms
0     patient       3062
1      diabet       2764
2         +/-       1875
3     insulin       1785
4        less       1481
5       level       1448
6     control       1382
7           p       1354
8       group       1342
9       studi       1286
Category  C13
  Categoty id  top terms
0     patient       1913
1       women       1769
2    pregnanc       1388
3         use        818
4       fetal        786
5       group        778
6      infect        706
7       studi        697
8   treatment        664
9         may        616
Category  C12
  Categoty id  top terms
0     patient       5666
1       renal       3372
2   treatment       1641
3    patients      

## Conclusion and summary

After analysing the lists of top 10 frequent words in each category we can see that there are words
that appeare many times in mane categories.
We can remove these words and gain more accurecy.
In PART 2 of the assignemt we will remove there words from our corpus.
The frequent words that apprear in many categoried and we will remove are:
- patient
- infect
- case
- use
- study / studies
- cell
- treatment
- increase
- '+/-'
- group
- p 
- result


## Explain the expected challenges

The challenges we expect to meet are:
- Terms that are common in all categories may cause mistakes in fitting the models.
- Dealing with technical issues such as getting to know python and the different libraries
- Improving the algorithm in order to increase accuracy may be complicated, there might be parameters we didn't learn how to handle, and that can affect on our results.
- Exploring and checking what are the best algoritms for problem of this kind (classification of short documents).


<center style="color:#1E90FF">
    <h1>
2. Document classification
    </h1>
</center>

### new stop words list according to the findings from part 1:

In [61]:
extended_stop_words = stopwords.words('english') + list(string.punctuation)
extended_stop_words += ['patient', 'infect', 'case' , 'use', 'study','studies', 'cell', 'treatment', 'increase','+/-', 'group', 'p','result']

# Classify using Machine Learning

### We chose to clasify the documents by using Tf-Idf and Ngram as features Extraction Method

Extracting features from the training data using TF-IDF - Since we used 'TfidfVectorizer' it combines all the options of CountVectorizer and TfidfTransformer in a single model, so these are 2 feature extraction in one command.

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words=extended_stop_words)
feature_extraction_train = vectorizer.fit_transform(train_set.data)
print (feature_extraction_train.toarray())

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


Use features extraction from the test data using the same vectorizer

In [63]:
feature_extraction_test = vectorizer.transform(test_set.data)

print (feature_extraction_test.toarray())

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


## Classify

In the following methode we go over the given algorithm 'clf', classifying, and saving the result

In [64]:
result_display = []

def benchmark(clf, name):
    print('_' * 80)
    print("Training: ")
    print(clf)
    
    clf.fit(feature_extraction_train, train_set.target)

    pred = clf.predict(feature_extraction_test)

    score = metrics.accuracy_score(test_set.target, pred)
    result_display.append([name, 'Naive run', score])
    print ('accuracy: %f' %score)


### Naive Bayes

In [65]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

benchmark(MultinomialNB(), 'Naive Bayes')

________________________________________________________________________________
Training: 
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
accuracy: 0.252258


### KNN

In [66]:
from sklearn.neighbors import KNeighborsClassifier

benchmark(KNeighborsClassifier(), 'KNN')

________________________________________________________________________________
Training: 
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
accuracy: 0.398335


### SVM

In [67]:
from sklearn.linear_model import SGDClassifier


benchmark(SGDClassifier(), 'SVM')

________________________________________________________________________________
Training: 
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
accuracy: 0.445221


<h2>Conclusion:</h2>
<p>
We can see that the best accuracy achived by the SVM - SGDClassifier algorithm
</p>
<h2>Try to Improve Accurecy:</h2>
<p>
We will try to improve it using Pipeline and tune the algorithm and feature extraction parameters:
</p>

In [68]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

nb_clf = Pipeline([('vect', vectorizer),('clf', MultinomialNB())])
knn_clf = Pipeline([('vect', vectorizer),('clf', KNeighborsClassifier())])
sgd_clf = Pipeline([('vect', vectorizer),('clf', SGDClassifier())])

Lets find the best parameters for each model

### Naive biase

In [25]:
parameters =  {'vect__max_df': (0.3,0.4,0.5,0.6,0.7, 0.75, 1.0),
               'vect__ngram_range': ((1, 1), (1, 2)),# unigrams or bigrams
               'clf__alpha': (0.0001, 0.01,1.0), 
               'clf__fit_prior':[True, False] }

naive_clf = GridSearchCV(nb_clf, parameters)
naive_clf = naive_clf.fit(train_set.data, train_set.target)

print('Best score: ',naive_clf.best_score_)
print('Best params: ',naive_clf.best_params_)


('Best score: ', 0.26924182881242215)
('Best params: ', {'vect__ngram_range': (1, 1), 'vect__max_df': 0.6, 'clf__fit_prior': False, 'clf__alpha': 0.01})


The result in the folowing photo:

![naive biase results](./images/nb.png)

### KNN

In [112]:
knn_parameters = {'vect__max_df': (0.3,0.4,0.5,0.6,0.7, 0.75, 1.0),
               'vect__ngram_range': ((1, 1), (1, 2)),# unigrams or bigrams
               'clf__algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute'), 
               'clf__leaf_size': (10,20,30,40),
               'clf__n_neighbors': (2,5,7),
               'clf__weights': ['uniform', 'distance']}

knn_gs = GridSearchCV(knn_clf, knn_parameters)
knn_gs = knn_gs.fit(train_set.data, train_set.target)
print('Best score: ',knn_gs.best_score_)
print('Best params: ',knn_gs.best_params_)

('Best score: ', 0.36068244991852777)
('Best params: ', {'vect__ngram_range': (1, 2), 'clf__algorithm': 'auto', 'vect__max_df': 0.6, 'clf__leaf_size': 10, 'clf__weights': 'uniform', 'clf__n_neighbors': 7})


The result is shown in the following photo:

![knn results](./images/knn.png)

### SVM

In [ ]:
parameters =  {'vect__max_df': (0.3,0.4,0.5,0.6,0.7, 0.75, 1.0),
               'vect__ngram_range': ((1, 1), (1, 2)),# unigrams or bigrams
               'clf__alpha': (0.0001, 0.01,1.0), 
               'clf__loss':['hinge','epsilon_insensitive','modified_huber'],
               'clf__penalty': ('l2', 'elasticnet'),
               'clf__average':[False, True]}

gs_clf = GridSearchCV(sgd_clf, parameters)
gs_clf = gs_clf.fit(train_set.data, train_set.target)
print('Best score: ',gs_clf.best_score_)
print('Best params: ',gs_clf.best_params_)

The results shown in a photo:

![svm results](./images/svm.png)

# Classify using the best parameters

### Naive biase

In [69]:
vectorizer = TfidfVectorizer(ngram_range = (1,1), sublinear_tf=True, max_df=0.6, stop_words=extended_stop_words)

improved_features_train = vectorizer.fit_transform(train_set.data)
improved_features_test = vectorizer.transform(test_set.data)

clf = MultinomialNB(fit_prior=False,  alpha = 0.01)

clf.fit(improved_features_train, train_set.target)
pred = clf.predict(improved_features_test)

score = metrics.accuracy_score(test_set.target, pred)
result_display.append(['Naive Bayes', 'Improved', score])

print("improved accuracy:   %0.3f" % score)

improved accuracy:   0.406


### KNN

In [70]:
vectorizer = TfidfVectorizer(ngram_range = (1,2), sublinear_tf=True, max_df=0.6, stop_words=extended_stop_words)

improved_features_train = vectorizer.fit_transform(train_set.data)
improved_features_test = vectorizer.transform(test_set.data)

clf = KNeighborsClassifier(n_neighbors=7, weights='uniform', algorithm='auto',leaf_size=10)

clf.fit(improved_features_train, train_set.target)
pred = clf.predict(improved_features_test)

score = metrics.accuracy_score(test_set.target, pred)
result_display.append(['KNN', 'Improved', score])

print("improved accuracy:   %0.3f" % score)

improved accuracy:   0.416


### SVM 

In [71]:
vectorizer = TfidfVectorizer(ngram_range = (1,2), sublinear_tf=True, max_df=0.7, stop_words=extended_stop_words)

improved_features_train = vectorizer.fit_transform(train_set.data)
improved_features_test = vectorizer.transform(test_set.data)

clf = SGDClassifier(average=True, loss="modified_huber", penalty = 'elasticnet', alpha = 0.0001)

clf.fit(improved_features_train, train_set.target)
pred = clf.predict(improved_features_test)

score = metrics.accuracy_score(test_set.target, pred)
result_display.append(['SVM', 'Improved', score])

print("improved accuracy:   %0.3f" % score)

improved accuracy:   0.218


Let's display the results and analyze them

In [72]:
df = pd.DataFrame(result_display, columns=['Algorithm name', 'Type of run', 'Accuracy score'])

df

,Algorithm name,Type of run,Accuracy score
0,Naive Bayes,Naive run,0.252258
1,KNN,Naive run,0.398335
2,SVM,Naive run,0.445221
3,Naive Bayes,Improved,0.406346
4,KNN,Improved,0.416477
5,SVM,Improved,0.217938


We can see that the SVM model has returned the best result, and that was even before using the improved parameters.
The meaning is that even after trying to tune the different parameters, the 'Naive run' for SVM has returned the best result in 0.443 accuracy rate.

### The main challenge

We can see that the accuracy in all the algorithms is not the as high as we accepted to receive. Since it took a lot of time to use the GridSearch, it was very disappointing to see the accuracy is lower then expected. We assume the reason is that the result we got was because the parameters were overfitting to the train set, and therefore in the real-time test the prediction to the test set wasn't good.

### Task challenges and effective solutions

##### Challenges

* Finding the parameters to improve the algorithms took a lot of time 
* Arrange all the information we found in a nice way
* The results of prediction with the tuned parameters were very low
* Understand the best way to avoid 'Overfitting'

##### Effective solutions

* The train set should include a variaty if information so that 'Overfitting' will be prevented
* Use strong and multiple computers to avoid wating time on computing